https://www.geeksforgeeks.org/working-with-pdf-files-in-python/

In [84]:
import re
import pandas as pd
from pypdf import PdfReader

In [85]:
reader = PdfReader("asda_statements/February 2024.pdf")
page = reader.pages[0] 
text = page.extract_text()

In [86]:
date_pattern        = r"(\d{1,2}\s*[a-zA-Z]{3})"
description_pattern = r"([^£]+)"
money_pattern       = r"£(\d+\.\d{2})"
row_pattern = r"\s+".join([date_pattern, date_pattern, description_pattern, money_pattern])

text = re.sub(r"\n", " ", text)
rows = re.findall(row_pattern, text)

In [87]:
col_names = ("transaction_date", "posting_date", "description", "amount")
df = pd.DataFrame(rows, columns=col_names)

In [92]:
df.dtypes

transaction_date    object
posting_date        object
description         object
amount              object
dtype: object

In [90]:
df["amount"] = pd.to_numeric(df.amount)

0     5.50
1     3.55
2     1.58
3    11.94
4     6.50
5    12.95
6    25.75
7    12.95
8     6.50
Name: amount, dtype: float64

In [91]:
df

,transaction_date,posting_date,description,amount
0,22 Jan,23 Jan,TFL TRAVEL CH TFL.GOV.UK/CP,5.50
1,24 Jan,25 Jan,ALDI 71 775 READING,3.55
2,27 Jan,29 Jan,ALDI 71 775 READING,1.58
3,27 Jan,29 Jan,ALDI 71 775 READING,11.94
4,29 Jan,30 Jan,SUMUP *SOUTH READING JUDREADING,6.50
5,30 Jan,31 Jan,ALDI 71 775 READING,12.95
6,01 Feb,02 Feb,TFL TRAVEL CH TFL.GOV.UK/CP,25.75
7,02 Feb,05 Feb,ALDI 71 775 READING,12.95
8,05 Feb,06 Feb,SUMUP *SOUTH READING JUDREADING,6.50
